In [ ]:
import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot
import netCDF4

import sys
sys.path.append('../extern/fabm-mizer/python')
import mizer

In [ ]:
path = r'../datasets/NEMO-MEDUSA-Agulhas.nc'
with netCDF4.Dataset(path) as nc:
    print('%s contains: %s' % (path, ', '.join(nc.variables.keys())))
    temp = nc['thetao'][:, 0, 0]
    ZME = nc['ZME'][:, 0, 0]
    ZMI = nc['ZMI'][:, 0, 0]
    PHD = nc['PHD'][:, 0, 0]
    PHN = nc['PHN'][:, 0, 0]
    depth = nc['bm_int'][:, 0, 0]**2 / nc['bm2_int'][:, 0, 0]
    numtime = nc['time_counter'][:]
    time = netCDF4.num2date(numtime, nc['time_counter'].units, only_use_cftime_datetimes=False)

In [ ]:
fig, (ax1, ax2, ax3) = pyplot.subplots(nrows=3, figsize=(12,8), sharex=True)
ax1.plot(time, temp)
ax1.grid()
ax1.set_ylabel('experienced temperature (°C)')

ax2.plot(time, depth)
ax2.grid()
ax2.set_ylabel('interaction depth (m)')

ax3.plot(time, PHN, label='non-diatom phytoplankton')
ax3.plot(time, PHD, label='diatoms')
ax3.plot(time, ZMI, label='microzooplankton')
ax3.plot(time, ZME, label='mesozooplankton')
ax3.grid()
ax3.set_ylabel('concentration (mmol N/m³)')
ax3.set_xlabel('time')
ax3.legend();

In [ ]:
w_min = 1e-3
w_inf = 1e6

# Parameters of the size spectrum model (mizer, http://dx.doi.org/10.1111/2041-210X.12256)
parameters = {
    'w_min': w_min,
    'w_inf': w_inf,
    'nclass': int(100 * (numpy.log10(w_inf) - numpy.log10(w_min)) / 9.),
    'T_dependence': 1,
    'T_ref': 13.,
    'E_a': 0.63,
    'beta': 100,
    'sigma': float(numpy.log(10.)),
    'gamma': 156,
    'q': 0.82,
    #'n': 0.82, #0.93,
    'alpha': 0.2,
    'z0_type': 1,
    'z0pre': 0.1,
    'z0exp': -0.25,
    'w_s': 1000.,
    'z_s': 0.3,
    'ks': 0.,
    'SRR': 0,
    'recruitment': 0.,
    'h': 1e9, #30
    'fishing_type': 1,
    'w_minF': 1.25, # Blanchard et al 2012
    'F': 0.4 #4 #8  # note: need to put double the intended value due to fisheries equation!
}

# Function for converting from Equivalent Spherical Diameter (micrometer) to wet mass in g
def esd2mass(d): # d: equivalent spherical diameter in micrometer
    V = 4./3. * numpy.pi * (numpy.array(d) / 2e6)**3  # V: volume in m3
    return V*1e6  # mass in g approximately equals volume in m3 multiplied by 1e6 (assumes density of 1000 kg/m3)

w_esd2, w_esd20, w_esd200 = esd2mass([2., 20., 200.])

scale_factor = 10 * 0.001 * 12 * 106. / 16. # 10 g wet mass/g carbon * 0.001 g C/mg C * 12 mg C/mmol C * 106/16 mmol C/mmol N
prey = []
for long_name, w_range, ncname in [
    ('diatoms',                  (w_esd20, w_esd200), 'PHD'),
    ('non-diatom phytoplankton', (w_esd2,  w_esd20),  'PHN'),
    ('microplankton',            (w_esd20, w_esd200), 'ZMI'),
    ('mesoplankton',             (1e-5,    1e-3),     'ZME')
]:
    timeseries = mizer.datasources.TimeSeries(path, ncname, x=0, y=0, scale_factor=scale_factor, time_name='time_counter', allow_mask=True)
    prey.append(mizer.Prey(long_name, w_range, timeseries))
prey_collection = mizer.PreyCollection(*prey)
prey_collection = mizer.GriddedPreyCollection(prey_collection)

# Environmental conditions
temp = mizer.datasources.TimeSeries(path, 'thetao', x=0, y=0, time_name='time_counter', allow_mask=True)
depth = mizer.datasources.TimeSeries(path, 'bm_int**2/bm2_int', x=0, y=0, time_name='time_counter', allow_mask=True)

# create mizer model
m = mizer.Mizer(prey=prey_collection, parameters=parameters, temperature=temp, depth=depth, recruitment_from_prey=2)

In [ ]:
# Time-integrate
result = m.run(temp.times, spinup=50, verbose=True, save_spinup=False, save_loss_rates=True, save_f=True)

In [ ]:
from ipywidgets import widgets
def show_spectrum(day):
    result.plot_spectrum(day, normalization=0, global_range=True)
slider = widgets.interact(show_spectrum, day=(0, len(result.t)-1))

In [ ]:
result.plot_biomass_timeseries()
result.plot_timeseries('landings')
result.plot_annual_mean('landings', plot_change=True)

In [ ]:
anim = result.animate_spectrum(normalization=0)